In [13]:
# load train.txt
with open("dataset/train.txt") as f:
    # declare an empty list for later saving train data values
    train_value = []
    predict_value = []

    # read the train values line by line
    for line in f:
        # seperate every sequence number by comma
        sep_string = line.split(',')

        count = 0

        # using slice of list to get the value we acctually need
        while(count < 2):
            # first and second number sequence need the last three digits
            sep_string[count] = sep_string[count][-3:]
            count += 1
        
        # the third number sequence need last four digits and no '\n'
        sep_string[2] = sep_string[2][-5:-1]
        train_value.append(sep_string[:2])
        predict_value.append(sep_string[-1])

In [19]:
count = 0
while count < len(train_value):
    train_value[count] = list(map(int, train_value[count]))
    count += 1

predict_value = list(map(int, predict_value))

print(train_value[0], type(train_value[0][0]))
print(predict_value[0], type(predict_value[0]))

[686, 617] <class 'int'>
4983 <class 'int'>


In [49]:
from numpy import array


split_percent = 0.80
split = int(split_percent*len(train_value))

# format as NumPy arrays
X, y = array(train_value[:split]), array(predict_value[:split])
val_X, val_y = array(train_value[split:]), array(predict_value[split:])

largest = max(y)
val_largest = max(val_y)

n_numbers = 2

# normalize
X = X.astype('float') / float(largest * n_numbers)
y = y.astype('float') / float(largest * n_numbers)

val_X = val_X.astype('float') / float(val_largest * n_numbers)
val_y = val_y.astype('float') / float(val_largest * n_numbers)

print(largest)
print(len(X))
print(len(val_X))

7480
200000
50000


In [30]:
# invert normalization
def invert(value, n_numbers, largest):
	return round(value * float(largest * n_numbers))

print(invert(y[0], 2, largest))

4983


In [85]:
import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense

# create LSTM
model = Sequential()
model.add(LSTM(6, input_shape=(n_numbers, 1), return_sequences=True))
model.add(LSTM(2))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

In [86]:
epochs = 20
n_batch = 256
n_examples = split

# train LSTM
for _ in range(epochs):
	X = X.reshape(n_examples, n_numbers, 1)
	model.fit(X, y, epochs=1, batch_size=n_batch, verbose=1)

Epoch 1/1
200000/200000 [==============================] - 2s 12us/step - loss: 0.0132
Epoch 1/1
200000/200000 [==============================] - 2s 9us/step - loss: 0.0074
Epoch 1/1
200000/200000 [==============================] - 2s 9us/step - loss: 0.0022
Epoch 1/1
200000/200000 [==============================] - 2s 8us/step - loss: 2.5606e-05
Epoch 1/1
200000/200000 [==============================] - 2s 9us/step - loss: 2.0735e-05
Epoch 1/1
200000/200000 [==============================] - 2s 9us/step - loss: 1.5853e-05
Epoch 1/1
200000/200000 [==============================] - 2s 8us/step - loss: 1.1164e-05
Epoch 1/1
200000/200000 [==============================] - 2s 8us/step - loss: 7.2147e-06
Epoch 1/1
200000/200000 [==============================] - 2s 8us/step - loss: 4.3238e-06
Epoch 1/1
200000/200000 [==============================] - 2s 9us/step - loss: 2.6590e-06
Epoch 1/1
200000/200000 [==============================] - 2s 9us/step - loss: 1.6757e-06
Epoch 1/1
200000/2000

In [87]:
# evaluate on validation data
val_numbers = len(train_value) - n_examples
val_X = val_X.reshape(val_numbers, n_numbers, 1)
# print(val_X)
val_result = model.predict(val_X, batch_size=n_batch, verbose=1)

50000/50000 [==============================] - 1s 10us/step


In [103]:
from math import sqrt
from sklearn.metrics import mean_squared_error

# calculate validation error
expected = [invert(x, n_numbers, val_largest) for x in val_y]
predicted = [invert(x, n_numbers, val_largest) for x in val_result[:,0]]
rmse = sqrt(mean_squared_error(expected, predicted))

print('RMSE: %f' % rmse)

RMSE: 5.504769


In [89]:
# show some examples
for i in range(20):
	error = expected[i] - predicted[i]
	print('Expected = %d, Predicted = %d (err = %d)' % (expected[i], predicted[i], error))

Expected = 1363, Predicted = 1371 (err = -8)
Expected = 1962, Predicted = 1967 (err = -5)
Expected = 6813, Predicted = 6814 (err = -1)
Expected = 4184, Predicted = 4188 (err = -4)
Expected = 2029, Predicted = 2030 (err = -1)
Expected = 1354, Predicted = 1361 (err = -7)
Expected = 5371, Predicted = 5376 (err = -5)
Expected = 5614, Predicted = 5619 (err = -5)
Expected = 5568, Predicted = 5577 (err = -9)
Expected = 3746, Predicted = 3751 (err = -5)
Expected = 6224, Predicted = 6231 (err = -7)
Expected = 4956, Predicted = 4960 (err = -4)
Expected = 2650, Predicted = 2649 (err = 1)
Expected = 2511, Predicted = 2510 (err = 1)
Expected = 2965, Predicted = 2964 (err = 1)
Expected = 3079, Predicted = 3078 (err = 1)
Expected = 3432, Predicted = 3432 (err = 0)
Expected = 2523, Predicted = 2522 (err = 1)
Expected = 3842, Predicted = 3844 (err = -2)
Expected = 5549, Predicted = 5556 (err = -7)
